In [139]:
from pcapfile import savefile
import scapy
from scapy.all import *
from scapy.utils import PcapReader
from scapy.layers.inet import IP, TCP
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
import pandas as pd
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
spark = SparkSession.builder \
     .master("local") \
     .appName("test 2") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

22/09/15 15:50:12 WARN Utils: Your hostname, MacBook-Pro-Ekaterina.local resolves to a loopback address: 127.0.0.1; using 192.168.0.133 instead (on interface en0)
22/09/15 15:50:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 15:50:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data = "traff_test.pcap"

In [6]:
testcap = rdpcap('traff_test.pcap')

In [385]:
#testcap.show()

In [9]:
len(testcap) #frames

66209

In [45]:
pkts = 0
flows = 0
ft = {}
for pkt in PcapReader(data):
            pkts += 1
            if IP in pkt:
                if TCP in pkt:
                    ft[(pkt[IP].src,pkt[IP].dst,pkt[TCP].sport,pkt[TCP].dport)] = pkt.wirelen 
                    flows +=1                

66209


In [387]:
#ft #IP-адрес источника, IP-адрес назначения, номер порта источника и номер порта назначения, а также длина пакета

In [13]:
dept = []
for key, value in ft.items():
    temp = list(key)
    temp.append(value)
    dept.append(temp)

In [54]:
len(dept)

790

In [230]:
df = pd.DataFrame(dept)
df.rename(columns = {0 : 'Source', 1 : 'Destination', 2: 'Source port', 3: 'Destination Port', 4: 'Bytes'}, inplace = True)
df.head(3)

,Source,Destination,Source port,Destination Port,Bytes
0,116.202.225.111,192.168.0.120,443,49674,101
1,192.168.0.120,116.202.225.111,49674,443,326
2,152.199.19.161,192.168.0.120,443,49795,60


In [50]:
df.shape

(790, 5)

In [15]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [181]:
sparkDF = spark.createDataFrame(data=df)

In [183]:
sparkDF.write.parquet("network_data.parquet") 

In [249]:
parDF1 = spark.read.parquet("network_data.parquet")

In [250]:
parDF1.show(3)

+---------------+---------------+-----------+----------------+-----+
|         Source|    Destination|Source port|Destination Port|Bytes|
+---------------+---------------+-----------+----------------+-----+
|116.202.225.111|  192.168.0.120|        443|           49674|  101|
|  192.168.0.120|116.202.225.111|      49674|             443|  326|
| 152.199.19.161|  192.168.0.120|        443|           49795|   60|
+---------------+---------------+-----------+----------------+-----+
only showing top 3 rows



In [244]:
parDF1.registerTempTable("data_net")
spark.sql('''
SELECT COUNT(*) AS count_cols FROM data_net
''').toPandas().head(1)

,count_cols
0,790


In [295]:
count_source = spark.sql('''
SELECT Source, COUNT(Source) as Count
FROM data_net
GROUP BY Source
ORDER BY Count DESC
''').toPandas()
count_source.head(5)
#количество записей сгруппированных по Source

,Source,Count
0,192.168.0.120,386
1,192.168.0.1,279
2,2.21.193.110,15
3,188.234.145.146,14
4,188.234.145.155,13


In [188]:
spark.sql('''
SELECT COUNT(DISTINCT Source)
FROM data_net
''').toPandas() #всего записей сгруппированных по Source

,count(DISTINCT Source)
0,61


In [189]:
groupBySourceDestCount = parDF1.groupBy("Source", "Destination") \
      .agg(count("Source").alias("CountSourceDest")) \
      .show(5)
#количество записей сгруппированных по Source и Destination

+---------------+--------------+---------------+
|         Source|   Destination|CountSourceDest|
+---------------+--------------+---------------+
|  173.194.73.95| 192.168.0.120|              1|
|  192.168.0.120| 51.105.71.137|              1|
|   2.21.193.110| 192.168.0.120|             15|
|188.234.145.155| 192.168.0.120|             13|
|  192.168.0.120|216.239.32.116|              1|
+---------------+--------------+---------------+
only showing top 5 rows



In [190]:
groupBySourceDestCount = parDF1.groupBy("Source", "Destination").sum() \
      .agg(count("Source").alias("CountSourceDest")) \
      .show()
#всего записей сгруппированных по Source и Destination

+---------------+
|CountSourceDest|
+---------------+
|            114|
+---------------+



In [191]:
SumBytesGroupBySource = parDF1.groupBy("Source") \
          .agg(sum("Bytes").alias("sum_bytes")) \
          .show(3)
#сумма по Bytes для группировки по Source

+-------------+---------+
|       Source|sum_bytes|
+-------------+---------+
| 142.251.1.97|      120|
|  20.82.228.9|       60|
|51.105.71.137|       60|
+-------------+---------+
only showing top 3 rows



In [275]:
sum_bytes = spark.sql('''
SELECT Destination, SUM(Bytes) as Bytes
FROM data_net
GROUP BY Destination
ORDER BY Bytes DESC
''').toPandas()
sum_bytes.head(10)

,Destination,Bytes
0,192.168.0.120,29267
1,192.168.0.1,14526
2,2.21.193.110,810
3,188.234.145.146,756
4,188.234.145.155,702
5,74.125.131.102,432
6,116.202.225.111,326
7,173.194.73.132,324
8,64.233.161.94,164
9,20.42.65.84,162


In [274]:
SumBytesGroupByDest = parDF1.groupBy("Destination") \
          .agg(sum("Bytes").alias("sum_bytes")) \
          .show(3)
#сумма по Bytes для группировки по Destination

+-------------+---------+
|  Destination|sum_bytes|
+-------------+---------+
| 142.251.1.97|      108|
|  20.82.228.9|       54|
|51.105.71.137|       54|
+-------------+---------+
only showing top 3 rows



In [177]:
SumBytesGroupBySP = parDF1.groupBy("Source port") \
          .agg(sum("Bytes").alias("sum_bytes")) \
          .show(3)
#сумма по Bytes для группировки по Source Port

+-----------+---------+
|Source port|sum_bytes|
+-----------+---------+
|      49967|       54|
|      49983|       54|
|      50002|       54|
+-----------+---------+
only showing top 3 rows



In [179]:
SumBytesGroupByDP = parDF1.groupBy("Destination port") \
          .agg(sum("Bytes").alias("sum_bytes")) \
          .show(3)
#сумма по Bytes для группировки по Destination Port

+----------------+---------+
|Destination port|sum_bytes|
+----------------+---------+
|           49967|       60|
|           49983|       60|
|           50002|       60|
+----------------+---------+
only showing top 3 rows



In [ ]:
#бонусная задача

In [276]:
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import plotly.express as px

In [286]:
import plotly.graph_objects as go
fig = go.Figure([go.Bar(
            x=sum_bytes['Destination'][0:10], y=sum_bytes['Bytes'][0:10],
            text=sum_bytes['Bytes'][0:2].apply(str),
            textposition='auto')])
fig.update_layout(
    title='Sources with max Bytes',
    xaxis_title="Sources",
    yaxis_title="Bytes",
)
fig.show()

In [296]:
fig = go.Figure([go.Scatter(
    x=sum_bytes['Destination'][0:10],
    y=sum_bytes['Bytes'][0:10],
    mode='markers',
    marker=dict(
        color=[145, 140, 135, 130, 125, 120,115,110,105,100],
        size=[100, 90, 70, 60, 60, 60,50,50,40,35],
        showscale=True
        )
)])
fig.update_layout(
    title='Sources with max Bytes',
    xaxis_title="Sources",
    yaxis_title="Bytes",
)
fig.show()

In [298]:
labels = count_source['Source'][0:10]
values = count_source['Count'][0:10]
fig = px.pie(count_source, values=values, names=labels)
fig.update_layout(
    title='Most Popular Sources'
)
fig.show()